# Noughts and crosses
The goal of this project is to implement a simple AI that plays noughts and crosses (also known as tic-tac-toe) using the minimax algorithm. Ultimately it should print out a game where both players play perfectly; it should be a draw.

## Minimax background
The minimax algorithm is attributed to John von Neumann (1928, `Zur Theorie der Gesellschaftsspiele`), but its key features were described earlier by Émile Borel (1921, `La théorie du jeu et les équations intégrales à noyau symétrique`). It has also been sugested that Charles Babbage may have known about the algorithm. Irregardless of who formalised it, anyone who has ever played Draughts, Chess, Go, or countless other games, has used it. At any point in the game you choose your next move by thinking several ahead, and considering what you and your opponent will do to maximise their respective chances of winning. In the formal version 'several ahead' means all the way to the very end, where one of you has won.

Minimax is an algorithm of game theory (a term invented by von Neumann with Morgenstern), which formalises how to act when you are in competition with others (If you want to know more then `The Joy of Game Theory` by  `Presh Talwalkar` is a friendly introduction). The basic version introduced here is for finding the optimal move in _zero sum_ games that are _turn based_ with _two players_ where everything is _deterministic_ and _observable_, and you have enough computation/memory to exhaust the _search tree_. To define these terms:
 * zero sum: There are n points to be won and they are distributed between the players at the end of the game. In other words, there is no possibility of cooperation, and one players fortune is anther's loss. Any game where one player wins and the other loses satisfies this; a draw is half the points to each player.
 * turn based: Players take turns to make a move.
 * two player: There are two players!
 * deterministic: No dice. The consequence of any action is known, exactly.
 * observable: No secrets, so both players know everything.
 * search tree: This is every possible state the game can be in, and the transitions (moves) between them made by the players.

The minimax algorithm is solving the _Nash equilibrium_ (Its discovery is the subject of the film _A Beautiful Mind_ (2001)) for games with the above properties. Note that many of the above properties can be relaxed with more sophisticated versions. Even when it can't approximations are used: Alpha Go used minimax to beat homosapiens at Go, but never could have held the entire search tree of Go in memory; that would be more states than there are atoms in the universe... even after you bulked up by replacing every atom with a copy of the universe. Instead, it used machine learning to estimate what the probability of winning from any state was, so it didn't have to search to the end.

## Noughts and crosses explained ;)
(alternatively, tic-tac-toe)

Two players take turns to place symbols on a 3x3 grid. First one to make a line (horizontal, vertical, or diagonal) of three of their symbol wins. Tradition is that the crosses (`x`) go first, the noughts (`o`) second.

Here is an example game (rendered using ascii art, with only the vertical lines!):
```
 | |       | |      | |x     | |x     |x|x    o|x|x    o|x|x
 |x|  ->   |x|o ->  |x|o ->  |x|o ->  |x|o ->  |x|o ->  |x|o
 | |       | |      | |     o| |     o| |     o| |     o|x|
```
`x` ultimately wins with a vertical line. As for when `o` lost, they did so with their very first move, even though it took `x` a few moves to force the win.

## Minimax explanation
Minimax is best understood by starting at the end of a game. Imagine the board looks like this:
```
o|x|x
 |x|o
o|x|
```
The game has ended, and `x` has won. Lets say that the value is `+1` when `x` wins, `-1` when `y` wins, and `0` for a draw. So this state (of the board) has a score of `+1`.

Key to minimax is the realisation that if we can assign a score to the end states then we can assign a score to any other state by assuming the players always play the best move available. For `x` that means the move that maximises the score, for `o` that means the move that minimises the score.

Lets go back a move:
```
o|x|x
 |x|o
o| |
```
How do we assign a score? It is the turn of `x` and they have three choices:
```
o|x|x    o|x|x    o|x|x
x|x|o     |x|o     |x|o
o| |     o|x|     o| |x
```
If we assume the score is defined for each of these states then naturally `x` will play the move associated with ending in the highest scoring state.

We know the score for the middle state is `+1`, as it is a winning state. What about the other two?
The score of the left and right states can be calculated in the exact same way as for this state, the one difference being it will then be the turn of `o`, who will want to minimise the score.

The left state has two choices for `o`:
```
o|x|x    o|x|x
x|x|o    x|x|o
o|o|     o| |o
```
which will be immediately followed by `x` taking the only move it has left and ending the game. On the left this is a draw (score=`0`), and on the right a win for `x` (score=`+1`). `o` wants to minimise so it will choose the left choice - a draw is preferable to a loss. So the left state scores `0` when `x` looks at it, because `x` assumes `o` will do the best they can. The same argument holds for the right choice. So when `x` is taking its move it will always choose the middle option, because it has the highest score, corresponding to victory.

This pattern repeats. Going right back to the start of the game `x` will consider every possible game that can be played, all the way to the end. And by assuming that it always takes the best move, and the opposition always takes the best move avaliable to it (worst move for `x` because it's a zero sum game), it can calculate the score for every state as the minimum or maximum of the next move, depending on whose turn it is. The name of the algorithm, minimax, comes from this repeating pattern of selecting the minimum and maximum score.

## ASCII art - printing the state of game

It can be challenging to develop an algorithm without being able to see what it is doing. The below function visualises the state of the board, using the ascii-art style.

In [2]:
def print_state(state):
    """Prints the state."""
    for i in state:
        for j in range(len(i)):
            if j != (len(i)-1):
                print(i[j], end='|')
            else:
                print(i[j], end='')
        print()

# Code to test the above; output should be:
#  | | 
#  | | 
#  | | 
#
# o|x|x
#  |x|o
# o|x| 
#
# o|o|o
# x|x|x
#  | | 

print_state(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')))
print() # This adds a newline, so the outputs are not vertically next to each other

print_state((('o','x','x'),(' ','x','o'),('o','x',' ')))
print()

print_state((('o','o','o'),('x','x','x'),(' ',' ',' ')))

 | | 
 | | 
 | | 

o|x|x
 |x|o
o|x| 

o|o|o
x|x|x
 | | 


## Scoring the end

We need to be able to detect when a winning move has been played, and assign a score to it. The below function takes:

Input:
 * A board state, as specified above

Output:
 * If `x` has won `return 1`
 * If `y` has won `return -1`
 * If it is a draw `return 0`
 * If nobody has won, and moves can still be played, `return None`

In [3]:
def score_end(state):
    """Scores the game state if the game has ended, otherwise returns None."""
    #conditions to evaluate (in sequence)
    #1. if game is either won by x or o
    #2. if game is drawn
    #3. return None if game has moves left

    if (diagonal_check(state) + horizontal_vertical_check(state)) in [-1,1]:
        return (diagonal_check(state) + horizontal_vertical_check(state))
    elif game_is_drawn_check(state) == 0:
        return 0
    else:
        return None

def game_is_drawn_check(state):
    #check if game is drawn
    list_state = []
    for i in state:
        for j in i:
            list_state.append(j)
    
    if (list_state.count(' ') == 0) and (list_state.count('x') == 5 or list_state.count('o') == 5):
        return 0
 
def diagonal_check(state):
    result = 0
    check_diag_one = []
    check_diag_two = []
    for i in range(len(state)):
        for j in range(len(state[i])):
            if i == j:
                check_diag_one.append(state[i][j])
            if i+j == len(state[i])-1:
                check_diag_two.append(state[i][j])
    if check_diag_one.count('x') == 3 or check_diag_two.count('x') == 3:
        result = 1
    elif check_diag_one.count('o') == 3 or check_diag_two.count('o') == 3:
        result = -1        
    return result

def horizontal_vertical_check(state):
    result = 0
    check_list_hor = []
    check_list_ver = []
    for i in range(len(state)):
        for j in range(len(state[i])):
            check_list_hor.append(state[i][j])
            check_list_ver.append(state[j][i])
        if check_list_hor.count('x') == 3 or check_list_ver.count('x') == 3:
            result = 1
        elif check_list_hor.count('o') == 3 or check_list_ver.count('o') == 3:
            result = -1
        check_list_hor = []
        check_list_ver = []
    
    return result

# Code to test the above function...
print('Expected: None; Obtained:', score_end(((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))))
print('Expected: 1;    Obtained:', score_end((('o',' ','o'),('x','x','x'),(' ',' ',' '))))
print('Expected: 1;    Obtained:', score_end((('o','x','x'),(' ','x','o'),('o','x',' '))))
print('Expected: 1;    Obtained:', score_end((('o','x','x'),(' ','x','o'),('x','o',' '))))
print('Expected: -1;   Obtained:', score_end((('o','x','x'),(' ','x','o'),('o','o','o'))))
print('Expected: -1;   Obtained:', score_end((('o','x',' '),('o','x',' '),('o',' ','x'))))
print('Expected: -1;   Obtained:', score_end(((' ','x','o'),(' ','o','x'),('o',' ','x'))))
print('Expected: 0;    Obtained:', score_end((('o','x','o'),('x','x','o'),('o','o','x'))))

Expected: None; Obtained: None
Expected: 1;    Obtained: 1
Expected: 1;    Obtained: 1
Expected: 1;    Obtained: 1
Expected: -1;   Obtained: -1
Expected: -1;   Obtained: -1
Expected: -1;   Obtained: -1
Expected: 0;    Obtained: 0


## Playing a move

Given a state, as described above, and a move, e.g. 'place a `'x'` at row 2, column 1' the below function will return a __new__ state, with the move having been played.

Input:
 * State before the move.
 * Row to modify.
 * Column to modify.
 * What to place, `'x'` or `'o'`.

Output:
 * A new state, where only the given row/column has been modified.

In [4]:
def play(state, row, col, mark):
    """Returns a new state after the given move has been played."""
    
    #Intermediate variables
    convert_list = []
    intermediate_list = []
    convert_tuple = ()
    
    #for loop to convert tuple into list
    for i in state:
        convert_list.append(list(i))
        
    #updating list
    convert_list[row][col] = mark
    
    #coverting list back to tuple
    for i in convert_list:
        intermediate_list.append(tuple(i))
        convert_tuple = tuple(intermediate_list)
    return convert_tuple
        
# Code to test the above function - it should play the game given in the 'noughts and crosses' section above...
start = ((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))
move1 = play(start, 1, 1, 'x')
move2 = play(move1, 1, 2, 'o')
move3 = play(move2, 0, 2, 'x')
move4 = play(move3, 2, 0, 'o')
move5 = play(move4, 0, 1, 'x')
move6 = play(move5, 0, 0, 'o')
move7 = play(move6, 2, 1, 'x')

print_state(start)
print('')
print_state(move1)
print('')
print_state(move2)
print('')
print_state(move3)
print('')
print_state(move4)
print('')
print_state(move5)
print('')
print_state(move6)
print('')
print_state(move7)


# The assert statement throws an error if what it is testing is not true.
# The below assert statements confirm that the output of play is nested tuples.
# 'is' when applied to core types (such as tuple) ensures exact equivalence.
assert(type(move7) is tuple)
assert(type(move7[0]) is tuple)
assert(type(move7[1]) is tuple)
assert(type(move7[2]) is tuple)

 | | 
 | | 
 | | 

 | | 
 |x| 
 | | 

 | | 
 |x|o
 | | 

 | |x
 |x|o
 | | 

 | |x
 |x|o
o| | 

 |x|x
 |x|o
o| | 

o|x|x
 |x|o
o| | 

o|x|x
 |x|o
o|x| 


## Finding possible moves

Given a state we need to know all of the possible moves that the current player can make, that is, a list containing the coordinates of all empty places.

Input:
 * The state, as described above.
 
Output:
 * A list, containg a tuple for every possible move, where each tuple is `(row, column)`.

In [5]:
def moves(state):
    """Returns the list of moves that are avaliable from the current state."""
    result = []
    row_list = []
    col_list = []
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] == ' ':
                row_list.append(i)
                col_list.append(j)
    result = list(zip(row_list,col_list))
    return result

# Testing the above function. Note that a set is another Python data type;
# it's value here is it doesn't care about order when making comparisons, so if you
# output the list of moves in a different order it still does the right thing...
moves1 = moves(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')))
assert(set(moves1)==set([(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]))

moves2 = moves((('o',' ','o'),('x','x','x'),(' ',' ',' ')))
assert(set(moves2)==set([(0, 1), (2, 0), (2, 1), (2, 2)]))

moves3 = moves((('o','x','o'),('x','x','o'),('o','o','x')))
assert(moves3==[])

## Minimax

Now to implement the minimax algorithm; we will use the below function such that:

Input:
 * A game state.
 * Whose turn it is, either 'x' or 'o'.

Output:
 * The estimated score of the game.
 * `None` if the game has ended, or a tuple of `(row, column)`, the move the player is to take.

In [27]:
def score(state, player):
    """Given the game state and whose turn it is returns a tuple (estimated game score, best move to play)"""
        
    score_check = score_end(state)
    
    if score_check == 1 or score_check == -1 or score_check == 0:
        return score_check, None
    else:
        optimum_move = None
        optimum_score = None
    
        if player == 'x':
            optimum_score = -1
            next_player = 'o'
            
        if player == 'o':
            optimum_score = 1
            next_player = 'x'
        
        for move in moves(state):
            
            interm_score, interm_move = score(play(state, move[0], move[1],player), next_player)
            
            if player == 'x':
                if interm_score >= optimum_score:
                    optimum_score = interm_score
                    optimum_move = move
                
            if player == 'o':
                if interm_score < optimum_score:
                    optimum_score = interm_score
                    optimum_move = move
        
        return optimum_score, optimum_move
    
# Code to test the above...
print('Expected =  0; Obtained:', score(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')), 'x')[0])
print('Expected =  1; Obtained:', score(((' ',' ',' '),(' ','x','o'),(' ',' ',' ')), 'x')[0])
print('Expected =  1; Obtained:', score((('x',' ',' '),(' ','x','o'),(' ',' ',' ')), 'o')[0])
print('Expected = -1; Obtained:', score((('o',' ',' '),('x',' ',' '),('o',' ',' ')), 'o')[0])

Expected =  0; Obtained: 0
Expected =  1; Obtained: 1
Expected =  1; Obtained: 1
Expected = -1; Obtained: -1


## Bringing all together - A perfect game

Use the above functions to print out a 'perfect' game, using the minimax to select the move each time for both players (computer vs computer). The result should be a draw.

In [35]:
player= 'x'
state=((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))

is_moves_left = True
    
while is_moves_left:
    print("Current player: " + player)
    current_move = score(state, player)[1]
    next_state = play(state, current_move[0], current_move[1], player)
    state = next_state
    print_state(state)
    print()
        
    #checking the game status
    current_score = score_end(next_state)
    if current_score is not None:
        is_moves_left = False
        print("Game Over !!")
        if current_score == 1:
            print("Game is won by x")
        elif current_score == -1:
            print("Game is won by o")
        elif current_score == 0:
            print("It's a draw")
    else:
        print("Moves are left to be played ...")
        print()
        if player == 'x':
            player = 'o'
        else:
            player = 'x'

Current player: x
 | | 
 | | 
 | |x

Moves are left to be played ...

Current player: o
 | | 
 |o| 
 | |x

Moves are left to be played ...

Current player: x
 | | 
 |o| 
 |x|x

Moves are left to be played ...

Current player: o
 | | 
 |o| 
o|x|x

Moves are left to be played ...

Current player: x
 | |x
 |o| 
o|x|x

Moves are left to be played ...

Current player: o
 | |x
 |o|o
o|x|x

Moves are left to be played ...

Current player: x
 | |x
x|o|o
o|x|x

Moves are left to be played ...

Current player: o
o| |x
x|o|o
o|x|x

Moves are left to be played ...

Current player: x
o|x|x
x|o|o
o|x|x

Game Over !!
It's a draw


<b>As it can seen in the above cell, the game is a draw :)</b>